In [ ]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve credentials from environment variables
account = int(os.getenv("MT5_ACCOUNT"))
password = os.getenv("MT5_PASSWORD")
server = os.getenv("MT5_SERVER")

print(f"Account: {account}, Password: {password}, Server: {server}")
# Initialize MetaTrader 5
if not mt5.initialize():
    print("Failed to initialize MetaTrader5")
    quit()

# Login to the trading account
if not mt5.login(account, password=password, server=server):
    print(f"Failed to login to account {account}, error code: {mt5.last_error()}")
    mt5.shutdown()
    quit()

# Define parameters for historical data
symbol = "AUDUSD.iux"  # Replace with desired symbol
timeframe = mt5.TIMEFRAME_M5# Hourly timeframe
start_date = datetime(2025, 2, 1)  # Start date
end_date = datetime(2025, 3, 15)  # End date

# Check if symbol is available
if not mt5.symbol_select(symbol, True):
    print(f"Symbol {symbol} not found or unavailable, error code: {mt5.last_error()}")
    mt5.shutdown()
    quit()

# Retrieve historical OHLCV data
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)

if rates is not None and len(rates) > 0:
    # Convert to pandas DataFrame
    df = pd.DataFrame(rates)
    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'], unit='s')
    # Rename columns for clarity
    df = df[['time', 'open', 'high', 'low', 'close', 'tick_volume']]
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    # Set Date as index
    df.set_index('Date', inplace=True)
    # Round OHLC columns to 5 decimal places
    df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].round(5)
    print("\nHistorical OHLCV Data:")
    print(df.tail())  # Display last few rows for brevity
    
    # Save the DataFrame to a CSV file
    filename = f"{symbol}_M5_{start_date.strftime('%Y%m%d')}_{end_date.strftime('%Y%m%d')}.csv"
    df.to_csv(filename)
    print(f"Data saved to {filename}")
else:
    print(f"Failed to retrieve historical data for {symbol}, error code: {mt5.last_error()}")

# Shutdown MetaTrader 5
mt5.shutdown()

In [ ]:
df

In [ ]:
df.columns